In [1]:
#load the original ADR related People data
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
import pandas as pd

df = pd.read_csv('ADR_people.csv')

df = df.drop('Unnamed: 0', axis=1)

#load the ADR - itemnumber matrix

matrix= pd.read_csv('Prior_ITEM_NUMBER_Matrix.csv')

matrix = matrix.set_index('item_number')

# Normalize the matrix
normalized_matrix = (matrix - matrix.min()) / (matrix.max() - matrix.min())

# Replace all NaN values with 0
normalized_matrix = normalized_matrix.fillna(0)

#load the ADR - itemnumber normalised matrix
# normalized_matrix = pd.read_csv('Normalised_Prior_ITEM_NUMBER_Matrix.csv')

#load the ADR list

ADR_list = ['D52.1', 'D59.0','D59.2','D61.1', 'D64.2','D68.3','D69.5','E03.2',
            'E06.4','E15','E16.0','E23.1','E24.2','E27.3','E66.1','F11', 'F13',
            'F15', 'F19','F55','G21.0','G21.1','G21.2','G24.0','G25.0','G25.1',
            'G25.4','G25.6','G44.4','G62.0','G72.0','H26.3','H40.6','H91.0','I42.7',
            'I95.2','J70.2','J70.3','J70.4','K52.1','K71.1','K71.2','K71.6','K71.9',
            'K85.3','L10.5','L23.3','L24.4','L25.1','L27','L27.0','L27.1','L27.8',
            'L27.9','L43.2','L51','L51.1','L51.2','L51.8','L51.9','L56.0','L56.1',
            'L64.0','M10.2','M32.0','M34.2','M80.4','M81.4','M83.5','M87.1','N14.0',
            'N14.1','N14.2','N14.3','N14.4','N99.0','O35.5','O74.2','O74.3','O74.4',
            'P04.0','P04.1','P04.4','P58.4','P93','P96.1','P96.2','Q86.1','Q86.2',
            'R50.2','T36','T37','T38','T39','T39.1','T40','T40.3','T41','T42','T43',
            'T44','T45','T46','T46.0','T47','T48','T49','T50','T78','T78.2','T78.3',
            'T78.4','T78.8','T78.9','T80.1','T80.2','T80.3','T80.4','T80.5','T80.6',
            'T80.8','T80.9','T88.3','T88.6','T88.7','X40','X41','X42','X43','X44',
            'X45.5','Y40','Y41','Y42','Y42.7','Y43','Y44','Y44.0','Y44.2','Y45',
            'Y45.5','Y46','Y47','Y47.1','Y48','Y48.1','Y48.2','Y48.3','Y48.4','Y49',
            'Y50','Y51','Y52','Y53','Y54','Y55','Y55.1','Y56','Y57','Y57.5','Y57.9',
            'Y58','Y59','Y59.9','Y63','Y65.1','Y69','Y88.0','Z03.6']
# Create learning dataset and labels based on the n_days to be considered
import pandas as pd

def generate_dataset(df, normalized_matrix, ADR_list, n_days):
    df['create_datetime'] = pd.to_datetime(df['create_datetime'], format='%d/%m/%y')
    df['item_number'] = df['item_number'].str.strip()

    new_dataset = pd.DataFrame()
    labels = pd.DataFrame()
    index_list = []

    for person_id in df['person_id'].unique():
        person_df = df[df['person_id'] == person_id]
        person_adrs = person_df[person_df['item_number'].isin(ADR_list)]['item_number'].unique()
        if len(person_adrs) > 0:
            for adr in person_adrs:
                adr_date = person_df[person_df['item_number'] == adr]['create_datetime'].max()
                
                prior_period_df = person_df[(person_df['create_datetime'] < adr_date) &
                            (person_df['create_datetime'] >= adr_date - pd.DateOffset(days=n_days)) &
                            ~((person_df['item_number'] == adr) & (person_df['create_datetime'] == adr_date))]

                average_vector = normalized_matrix.loc[prior_period_df['item_number']].max()

                new_dataset = pd.concat([new_dataset, average_vector.to_frame().T], ignore_index=True)

                label_vector = pd.Series([1 if label_adr == adr else 0 for label_adr in ADR_list], index=ADR_list)

                labels = pd.concat([labels, label_vector.to_frame().T], ignore_index=True)

                index_list.append(f"{person_id}_{adr}")

    new_dataset.index = labels.index = index_list
    
        # Store the index of the rows with NaN values in new_dataset
    dropped_rows_index = new_dataset[new_dataset.isna().any(axis=1)].index

    new_dataset = new_dataset.dropna(axis=0)

    # Drop the corresponding rows from the labels dataset
    labels = labels.drop(index=dropped_rows_index)

    return new_dataset, labels

def labels_aggregate(labels):
    import pandas as pd

    # Assuming your labels dataframe is named 'labels'
    unique_first_letters = set([col[0] for col in labels.columns])

    # Create a new dataframe to store the combined columns
    combined_labels = pd.DataFrame(index=labels.index)

    for letter in unique_first_letters:
        # Find the columns with the same first letter
        matching_columns = [col for col in labels.columns if col[0] == letter]

        # Combine the matching columns into a single column
        combined_labels[letter] = labels[matching_columns].any(axis=1).astype(int)

    # Print the combined labels
    return combined_labels

# List of desired columns
desired_columns = ['Y', 'T', 'R', 'K', 'D', 'I', 'L', 'G', 'X', 'Z']

In [2]:
df

,person_id,age,gender,episode_id,drg_number,drg_version,item_number,create_datetime
0,0,104,1,168951,L63B,4.2,B96,2/7/04
1,0,104,1,168951,L63B,4.2,n39,2/7/04
2,0,104,1,168951,L63B,4.2,Z86,2/7/04
3,0,104,1,195656,f62b,4.2,I25.5,20/7/05
4,0,104,1,195656,f62b,4.2,I95.9,20/7/05
...,...,...,...,...,...,...,...,...
123994,1659,45,0,846115,K04A,6.2,92514-39,29/3/18
123995,1659,45,0,846115,K04A,6.2,E66.93,29/3/18
123996,1659,45,0,846115,K04A,6.2,K52.1,29/3/18
123997,1659,45,0,846115,K04A,6.2,U73.8,29/3/18


In [12]:
# Convert the 'item_number' column to string data type, because Word2Vec model accepts list of sentences (where each sentence is a list of words)
df['item_number'] = df['item_number'].astype(str)
df['create_datetime'] = pd.to_datetime(df['create_datetime'], format='%d/%m/%y')
df['item_number'] = df['item_number'].str.strip()

# Order by 'create_datetime' and group by 'person_id'
df_sorted = df.sort_values('create_datetime').groupby('person_id')['item_number'].apply(list).reset_index(name='item_list')


In [13]:
df_sorted

,person_id,item_list
0,0,"[B96, Z86, n39, Z86.43, I50.1, Z95.0, I25.5, I..."
1,1,"[5123, V1250, 8753, 4019, V1251, 412, 3123002,..."
2,2,"[I10, I48, i74, Y92, I50, Z92, Y54, I48, Z86, ..."
3,3,"[Y458, Z867, Z8711, N1890, N058, 3044500, I10,..."
4,4,"[42731, V4581, V1259, 53540, V1082, 7872, V104..."
...,...,...
1655,1655,"[Y92.9, Y57.9, T78.0]"
1656,1656,"[13706-05, 13706-03, 96199-02, D69.5, Z29.2, 9..."
1657,1657,"[Y43.4, I10, R11, R53, G25.1, E87.5, M32.1, Y9..."
1658,1658,"[W01.1, 50106-00, U73.9, 92514-99, 47459-01, 9..."


In [14]:
from gensim.models import Word2Vec

# Initialize the model
model = Word2Vec(vector_size=100, window=5, min_count=1, workers=4)

# Build the vocabulary
model.build_vocab(df_sorted['item_list'])

# Train the model
model.train(df_sorted['item_list'], total_examples=model.corpus_count, epochs=model.epochs)


(520561, 619995)

In [21]:
# Initialize new dataframe
new_df = pd.DataFrame(columns=['person_id', 'item_list', 'label'])

for idx, row in df_sorted.iterrows():
    # Find first ADR code in the item_list
    first_adr_idx = next((i for i, x in enumerate(row['item_list']) if x in ADR_list), None)
    print (first_adr_idx)
    
    if first_adr_idx is not None:
        # If an ADR is found, slice the list at the index of the first ADR
        adr_list = row['item_list'][:first_adr_idx]
        print (adr_list)
        new_df = new_df.append({'person_id': row['person_id'], 'item_list': adr_list, 'label': 1}, ignore_index=True)
    
    # Add original list as well, label as '0'  
    new_df = new_df.append({'person_id': row['person_id'], 'item_list': row['item_list'], 'label': 0}, ignore_index=True)


9
['B96', 'Z86', 'n39', 'Z86.43', 'I50.1', 'Z95.0', 'I25.5', 'I95.9', 'K31.8']
40
['5123', 'V1250', '8753', '4019', 'V1251', '412', '3123002', '4149', '5770', 'V4501', '4149', '1732', '1831', '3125501', 'V4501', '4019', '4149', '42731', 'V1250', '57410', '4019', '57410', 'C444', 'V4501', 'I10', '4545101', 'L910', 'I259', 'Z867', 'Z8643', 'Z950', 'C67.9', '368180-00', 'Z95.0', 'Z86.7', 'I10', 'Z86.43', 'I25.9', 'Z86.7', 'Z85.5']
6
['I10', 'I48', 'i74', 'Y92', 'I50', 'Z92']
186
['Y458', 'Z867', 'Z8711', 'N1890', 'N058', '3044500', 'I10', 'N058', 'K8011', 'Z867', 'N1890', 'I10', 'Z8711', 'K8011', 'R29.6', 'Z50.9', '95550-01', '95550-02', '95550-02', '95550-01', 'Z50.9', 'R29.6', 'Z50.9', '95550-01', '95550-02', 'R29.6', 'Z50.9', 'R29.6', '95550-02', '95550-01', 'R29.6', 'Z50.9', '95550-01', '95550-02', 'R29.6', 'Z50.9', '95550-02', '95550-01', 'Z50.9', '95550-01', 'R29.6', '95550-02', 'R29.6', 'Z50.9', '95550-01', '95550-02', '95550-01', '95550-02', 'R29.6', 'Z50.9', 'R29.6', '95550-02', 

38
['E11.51', 'E11.51', 'm79.86', 'Z86.43', 'E11.51', 'I70.2', 'm75', '48909', 'E1172', 'I200', 'I2511', 'E11.51', 'I70.21', 'E78.0', 'I10', 'E11.72', 'E11.5', 'R11', 'Z86.43', 'R55', 'J18.9', 'E11.72', 'Z50.9', 'U73.9', 'R09.0', 'S42.3', 'S01.1', 'Y92.22', 'Y92.09', 'Y45.0', 'W19', 'U73.8', 'Z86.43', '95550-09', '95550-10', 'E11.51', '95550-02', '95550-03']
48
['2E1.19', '2Z8.643', '2M1.71', '55500-3', '2M1.71', '2Z8.643', '2E1.19', '2Z9.665', '55500-2', '2Z5.09', 'E11.9', 'U82.3', 'G20', '92515-99', 'I48.9', '38350-00', '95550-03', 'E11.9', 'Z86.43', 'J18.9', '2I4.72', '2I2.511', '1Y9.222', '2Z9.50', '2E1.19', '1Y8.40', '1T8.10', '2U8.23', '2U8.01', '2I4.72', '2E1.19', '2U8.62', '2Z9.50', '2I0.81', '2U8.01', '2Z9.55', '2U8.62', '2U8.23', '2U8.01', '2Z9.50', '2I0.81', '2E1.19', 'R44.0', 'T83.89', 'Y92.24', 'E11.42', 'E11.51', '38477-01']
12
['59900', '38218', '92513', 'I25', 'i20', 'I51', 'Z72.0', 'R19.4', '92515-39', '42702-04', 'H26.9', 'U73.8']
75
['9013200', '3123001', 'J383', 'V1

42
['Z72.0', 'C82.0', 'c82', '92503-00', 'z51.1', 'z51.1', 'z51.1', 'C82.0', 'C82.0', 'C82.0', 'C82.0', '13915-00', 'z51.1', 'z51.1', 'C82.0', '13915-00', 'C82', '13915-00', 'C82.0', '13915-00', 'Z51.1', 'Z51.1', 'C82', '13915', 'Z51.', 'Z51.1', 'C82.5', 'C82.0', 'Z51.1', '13915-00', 'C82.0', 'Z51.1', 'C82.0', 'Z51.1', '13915-00', 'C02.0', 'Z51.1', '13915-00', '13915-00', 'C82.0', 'Z51.1', '13915-00']
14
['3034200', 'Z720', 'Z049', 'Z720', 'N603', '3209000', '13942', 'D62', 'T81', 'I80', 'B37', 'M21', 'Q68', 'm17']
35
['U82.3', 'I50.0', 'I48.9', 'U73.2', 'S49.9', 'K59.0', 'R94.5', 'N17.9', 'Y92.22', 'I51.8', '13400-00', '92514-30', 'W01.0', '30084-00', 'E11.29', 'I42.9', '95550-03', 'I50.0', 'I48.2', 'G41.8', 'Z73.8', 'G04.9', 'E87.7', '39000-00', 'Z03.0', 'F07.0', 'Y92.22', 'E11.21', 'F06.31', '95550-05', 'I83.1', 'F05.0', 'D86.8', 'N18.2', 'N17.9']
45
['8521', '6101', '33833-08', '90282-02', '92514-19', '30330-00', '95550-03', 'C77.4', '95550-12', 'T81.2', 'C43.9', 'U86.1', 'R19.4', 

14
['I48', '92514-39', '49558-00', 'M17.1', 'M23.22', 'M23.26', '49561-01', 'M71.2', 'Y54.3', 'Y52.5', 'Y52.1', 'Y92.22', 'U73.8', 'E87.7']
6
['45845-00', 'Z72.0', '92514-19', 'K08.1', '97234-00', '97236-00']
24
['92514-99', 'M23.89', 'Z86.43', 'M23.23', 'D04.3', '31235-00', 'Z86.43', '2I4.89', '2Z8.24', '2Z8.24', '2I4.81', '45003-01', '45665-01', '92515-19', '31340-00', 'C44.1', '42590-00', '2I4.89', 'Z86.43', 'I48.1', 'Z48.8', 'G81.9', 'I48.9', 'I61.9']
98
['92515-39', 'U73.8', 'K29.50', 'Y45.3', 'R10.4', 'K90.0', '30473-01', 'Y92.22', 'Z80.0', 'Z63.8', '96001-00', '96001-00', 'Z61.4', 'Z61.4', 'Z61.4', 'Z59.8', 'F32.20', 'Z61.4', 'Z59.8', 'Z59.8', 'F32.20', 'F32.20', 'Z59.8', '96001-00', 'Z59.8', 'Z63.8', 'F32.20', 'Z63.8', '96001-00', 'Z59.8', 'F32.20', '96001-00', 'Z63.8', 'Z61.4', 'Z61.4', 'F32.20', 'F32.20', 'Z61.4', '96001-00', 'Z61.4', 'Z63.8', 'F32.20', 'Z63.8', 'Z59.8', '96001-00', 'Z63.8', 'Z61.4', 'Z63.8', 'F32.20', 'Z63.8', 'Z59.8', 'Z59.8', 'Z63.8', 'Z59.8', '96001-00', 

0
[]
32
['59903-00', '38218-02', 'I25.11', 'Z86.43', 'I20.8', '92515-99', 'I10', '92209-01', '92514-40', '92514-49', 'E66.9', '95550-00', '38600-00', 'I25.11', '95550-03', 'E78.5', 'I97.8', 'E11.72', 'I35.1', '55118-00', 'J95.8', '38452-00', '38497-02', 'Y92.22', 'J98.1', 'N17.9', '38500-00', 'I95.9', 'J96.00', 'I10', 'E11.29', 'I31.3']
21
['53010', '4516', '53290', '53550', 'H04.5', '92503-00', '42623-00', '40300', '92514', '48651', 'm48', '96107', '95550', 'M51', '30445-00', 'K80.50', '92514-29', '95550-09', 'Y40.3', 'Y40.0', '95550-01']
0
[]
45
['481', '8192', '7231', '1827400', '5012401', 'M542', 'm51', 'M48', 'M54', 'm50', '18274', 'm54', 'M54.2', 'M54.2', '18274-00', 'M54.2', '92515-99', 'M54.2', '95550-10', '39013-00', '18274-02', '92515-99', '92515-99', 'M54.2', '39118-00', '95550-12', '39013-00', 'M54.5', 'R00.0', '95550-03', 'M54.92', '92515-39', '39118-00', 'M35.3', 'M13.99', 'M79.70', 'H83.0', 'M54.2', 'M79.70', 'Z50.9', '95550-03', 'M54.2', 'H83.0', '95550-02', '95550-12']

...

3
['T13.1', 'J45.9', 'T78.0']
2
['49518-00', 'Y92.22']
1
['R00.1']
3
['41861-00', '92514-99', 'J38.2']
4
['31230-04', 'E83.5', '92514-19', 'Y51.3']
6
['90469-00', '90466-02', '90472-00', 'O92.20', 'Z37.0', 'Z27.1']
1
['Y92.09']
122
['D05.1', '92514-29', 'C50.9', '31500-00', '45200-00', '31518-00', 'C50.8', '92514-29', 'C77.3', 'D05.1', '30300-00', 'C77.3', 'Z51.1', 'C50.9', '96199-00', 'C77.3', 'C50.9', '96199-00', 'Z51.1', 'Y92.22', 'D70', 'R50.9', 'Y43.3', 'C77.3', 'Z51.1', 'C50.9', '96199-00', 'Z51.1', 'C50.9', 'C77.3', '96199-00', 'Z45.2', '92514-99', 'Z51.1', 'C77.3', '13706-02', '34528-02', 'C50.9', '96199-00', '13706-02', 'B96.2', 'D64.9', 'D70', 'R50.9', 'N39.0', 'C50.9', 'C77.3', 'J22', 'Z53.0', '96199-00', 'C50.9', 'C77.3', 'Z51.1', 'C50.9', 'Z51.1', 'C77.3', '96199-00', 'C77.3', '96199-00', 'Z51.1', 'C50.9', 'C77.3', 'C50.9', '96199-00', 'Z51.1', 'J17.3', 'E87.6', 'B59', '95550-03', '13706-02', 'C50.9', 'C77.3', 'Z51.1', '96199-00', 'C50.9', 'C77.3', '96199-00', 'Z51.1', 'C5

3
['I84.1', '95550-03', 'J47']
5
['Z86.43', '32093-00', 'K62.1', 'R10.3', '92515-29']
5
['2D6.93', '2D6.93', '2D8.01', '37060-5', '2D6.93']
4
['92514-19', '45797-00', 'H91.8', 'Z97.4']
20
['O09.1', '92514-99', 'O03.4', '35643-03', '35640-03', 'O09.1', '92514-29', 'O02.1', 'O81', 'O36.5', '90465-05', 'R53', '90472-00', '90469-00', '92507-99', 'O12.0', '95550-03', 'O75.8', 'Z37.0', 'U80.2']
14
['Z86.43', 'N18.3', 'M31.6', 'I70.20', 'I10', 'H93.1', '95550-03', 'A09.9', 'J22', '95550-02', 'R10.4', '95550-03', 'Z50.9', 'J22']
4
['30216-01', '92514-99', 'L72.1', 'Y40.0']
27
['A09.9', '32090-01', 'R10.1', 'Z86.43', '30473-01', '92515-29', 'K29.50', '30688-00', 'C25.1', 'Z86.43', '30473-01', '92515-29', '30075-16', 'K66.0', 'C25.1', '92514-29', '30075-16', '30393-00', 'Y92.22', '30378-00', '30593-01', '33833-06', '95550-03', '92514-99', 'Z86.43', 'R50.9', 'Y83.6']
9
['92514-29', '13212-00', 'N97.9', 'E11.9', 'Z72.0', '95550-03', '30601-00', '30511-09', 'E66.8']
14
['L73.9', 'Y45.3', 'R94.5', '

0
[]
28
['41864-00', '92514-99', 'E11.72', 'E66.9', 'J38.1', 'Z86.43', 'D12.3', '92514-99', '32093-00', 'E11.9', 'Z85.0', 'Z08.9', 'D12.1', 'Z86.43', '2G4.59', '2I1.0', '2Z8.24', '2E1.172', '2Z8.643', '2E1.172', '2I2.08', '2I2.511', '2Z8.643', '2Z8.24', '2I1.0', 'U82.1', '95550-00', 'R00.0']
2
['A78', 'G47.0']
4
['41801-00', 'J35.2', '92514-99', 'Z88.8']
1
['J21.9']
6
['U79.3', 'R45.81', '95550-10', '95550-02', 'X60', 'U73.8']
1
['Y92.22']
9
['O80', 'O71.82', '95550-03', 'O70.0', 'Z37.0', 'I95.9', '90465-05', '92506-19', '90481-00']
13
['37821-00', '37417-00', 'Q54.4', '92514-19', '45206-08', 'Q54.0', '37815-00', 'N36.0', '92514-99', '45018-00', '37833-00', 'J45.9', 'T78.0']
15
['K57.30', '32093-00', 'K63.58', 'Z09.9', '92515-19', '95550-00', '39715-01', 'I95.9', 'D35.2', '92514-99', 'D32.0', 'Y92.22', 'E11.9', 'I10', '40803-00']
1
['H10.0']
12
['92514-99', '50104-00', 'E11.9', 'Z86.43', 'M20.1', 'M25.77', 'M65.97', '90574-00', '49833-00', 'Y92.22', 'Z86.43', '49518-00']
1
['T78.1']
58

6
['92514-29', 'M51.1', 'Y92.22', 'Z86.43', 'I95.9', 'R00.0']
2
['H65.9', 'Y92.22']
0
[]
6
['32138-00', '92515-99', 'K64.4', 'K64.9', '32093-00', 'R11']
18
['Z12.1', 'R10.4', '32090-00', '30473-01', '2G5.51', '2M5.11', 'Y45.0', 'Y92.22', 'Z72.0', 'K59.0', 'I95.9', '40300-00', '40330-00', '92514-99', '48684-00', '95550-03', 'R11', '48660-00']
1
['Y92.09']
12
['U72', 'Y92.2', 'S09.9', 'W04', 'F50.2', 'Z73.1', 'U73.8', 'U73.8', 'Y92.21', 'T43.69', 'F50.2', 'X60']
11
['Z12.9', '92515-19', '45206-09', '45515-00', 'C44.3', 'C78.2', 'J18.9', '95550-03', 'R59.0', 'Z86.43', 'Y92.22']
10
['P81.9', 'P70.4', 'P01.8', 'Z38.0', 'P08.1', '96199-02', 'B97.8', 'J21.8', 'R22', 'Y92.9']
114
['R56.8', 'Z72.0', 'F14.1', 'R10.1', 'F44.5', 'R11', 'Z72.0', '95550-00', 'R10.2', 'R11', 'N93.9', 'F44.5', '14203-00', '30075-37', 'N80.1', '35630-00', 'R10.2', '92514-19', '35637-10', '39323-00', '92515-99', 'G58.8', '92515-99', '39323-00', 'G58.8', '32093-00', 'Z72.0', 'K92.2', 'R12', 'K21.0', 'A09.9', '32090-00', 

1
['Y92.23']
19
['2Q3.915', '2Z3.83', '62020-7', '92044-00', 'P29.81', 'Q39.15', 'Q53.20', '96199-08', 'P25.1', '43843-01', '96199-09', '38806-00', '96199-02', '13319-00', '13882.02', '13706-02', '92062-00', 'P74.0', '92514-39']
1
['J22']
8
['U65.1', 'S22.5', '96026-00', 'S27.31', 'Y92.33', 'T78.1', 'S42.10', 'Y92.33']
3
['B97.8', '92062-00', '92514-99']
2
['Y92.9', 'T78.0']
29
['R10.1', 'U82.3', 'M54.6', '30473-01', 'U82.3', 'Z09.0', '32090-00', 'R10.3', 'K29.50', '92515-21', 'K22.8', 'Z87.12', '92515-39', 'Z03.8', 'Z87.18', '30473-00', '30688-00', 'Y92.24', 'U82.3', 'Y65.8', 'R11', 'T88.59', 'N40', 'R39.1', 'I95.9', 'N35.9', '92514-99', 'I97.89', '95550-12']
0
[]
12
['30514-01', '30512-03', '90942-01', '92514-39', 'I95.9', '95550-03', 'I97.8', 'E66.8', 'T88.5', 'U82.3', 'U83.3', 'Y83.9']
4
['Z38.0', 'Z03.70', 'J06.9', 'N39.0']
2
['X59', 'U73.2']
2
['P92.8', '96199-09']
2
['Y27.2', 'L27.2']
2
['Y92.9', 'U73.9']
4
['2H0.23', '2U8.62', '2N3.18', '2N1.38']
13
['Z37.0', 'Y92.22', '90481-0

In [22]:
new_df

,person_id,item_list,label
0,0,"[B96, Z86, n39, Z86.43, I50.1, Z95.0, I25.5, I...",1
1,0,"[B96, Z86, n39, Z86.43, I50.1, Z95.0, I25.5, I...",0
2,1,"[5123, V1250, 8753, 4019, V1251, 412, 3123002,...",1
3,1,"[5123, V1250, 8753, 4019, V1251, 412, 3123002,...",0
4,2,"[I10, I48, i74, Y92, I50, Z92]",1
...,...,...,...
3297,1657,"[Y43.4, I10, R11, R53, G25.1, E87.5, M32.1, Y9...",0
3298,1658,"[W01.1, 50106-00, U73.9, 92514-99, 47459-01, 9...",1
3299,1658,"[W01.1, 50106-00, U73.9, 92514-99, 47459-01, 9...",0
3300,1659,[U73.8],1


In [25]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Convert item lists to vectors
def convert_to_vector(item_list):
    if len(item_list) > 0:
        return np.mean([model.wv[word] for word in item_list], axis=0)
    else:
        return np.zeros(model.vector_size)  # return an empty vector if item_list is empty

new_df['vector'] = new_df['item_list'].apply(convert_to_vector)


In [1]:
new_df['label'] = new_df['label'].astype(int)

X = np.array(new_df['vector'].tolist())
y = new_df['label']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Train a random forest classifier on the training set
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# Predict the labels on the test set
y_pred = clf.predict(X_test)

# Print the accuracy
print('Accuracy:', accuracy_score(y_test, y_pred))


NameError: name 'new_df' is not defined